Following this course: https://github.com/louthy/language-ext/wiki/Thinking-Functionally    

In [ ]:
// quick setup
#r "nuget:LanguageExt.Core"
using LanguageExt;
using LanguageExt.ClassInstances;
using static LanguageExt.Prelude;
using static LanguageExt.List;
using static System.Console;

Installed Packages LanguageExt.Core, 4.2.9

### Starting simple
`AddOne :: int -> int`    
A function "maps" a value in the "domain" to a value in the "range". There is no "computation".

In [ ]:
int AddOne(int x) => x + 1;
return AddOne(5);

6

#### Important notes:    
`x` is not something that can change once we pass 5 in.    
This is not assignment - What is happening here is "binding".    
Once a domain value is "bound" is cannot be changed.     
`x` is a placeholder that can be referred to later, not changed.    
**There are no "variables", only values**

_side note, yes x is a variable being assigned a value - but it is forbidden 'functionaly'_

### Function Values
`Func<domain, range>`    

Similarly - the name `addOne` is just a "binding" to "the function that adds one to its input"    
"Every time you see the name `addOne`, replace it with the function that adds one to its input".    

For example, we can "bind" `addOne` to a new name.    
`plusOne` and `addOne` are "bound" to the same value, which is the function that adds one to its input.    
`plusOne :: int -> int`

In [ ]:
Func<int, int> plusOne = AddOne;
return (plusOne(5), AddOne(5));

Item1,Item2
6,6


### Simple Value
Imagine an operation that always returns `5` - this is a constant.    
in "C#" you can easily define that with the following: `const int C = 5;` or `static int C => 5; //Func<int>`.      
simple values do not need to be evaluated once "bound" - This is the subtle difference between them and function values, otherwise they are both values of Func.

With this, `5` and `five` are both just values that can be passed around. - This is a key part of thinking functionaly: functions are values that can be passed around to other functions.

A constant function for `5` would be defined as:    
`Func<Unit, int>`   
`five :: () -> int`

In [ ]:
const int C = 5;
//or
int C_alt => 5;
//or
Func<Unit, int> five = _ => 5;
//or
int Five(Unit _) => 5;

return (AddOne(C), AddOne(C_alt), AddOne(five(unit)), AddOne(Five(unit)));

// the presence of "Unit" is a bit annoying, this gets cleaned up later on.

Item1,Item2,Item3,Item4
6,6,6,6


### "Objects" vs "Values"

Values:
- A member of a domain. The domain of ints, strings, functions that map ints to strings, etc...    
- They are immutable
- They have no behaviour

Objects: 
- Encapsulation of data with associated behaviour
- Mutable

Avoid using "object" for standard values for functional C#.    
It should only be used to refer to true classes, or values that expose member functions.

### How types work with functions

Type notation: `domain -> range` == (C#) `Func<domain, range>`

"Higher-Order functions": A function that takes a function as a paramater, or returns another function.    
Think of LINQ `Select` or `Where`.

Example:
`EvalWith5ThenAdd2 :: (int -> int) -> int`

In [ ]:
int EvalWith5ThenAdd2(Func<int, int> fn) => 
    fn(5) + 2;

return EvalWith5ThenAdd2(AddOne);

8

In this function the domain is `(int -> int)` and the range is `int`.    
This means the input is a function that maps ints to ints, and the output is just an int.    

`timesThree :: int -> int`

In [ ]:
Func<int, int> timesThree = x => x * 3;

return EvalWith5ThenAdd2(timesThree);

17

Now, lets return a function rather than use one as input.    
`AdderGenerator :: int -> (int -> int)`

In [ ]:
Func<int, int> AdderGenerator(int numberToAdd) => 
    x =>  x + numberToAdd;

//or

Func<int, Func<int,int>> AdderGenerator_alt = numberToAdd => 
    x => x + numberToAdd;

// create new adders
var add1 = AdderGenerator(1);
var add2 = AdderGenerator_alt(2);

return (add1(1), add2(1));

Item1,Item2
2,3


### void?

How do we deal with something like:
```csharp
void PrintInt(int x)
{
    Console.WriteLine(x);
}
```
We cannot have `int -> void` since void is the absence of a value. But our function must have a range to map to.  Every function must have some output to map to.    
`Unit` to the rescue - it is like a `bool` which has two possible values `true`, `false`, though it only has one possible value `unit`.


In Funtional langauges the `Unit` type is often reffered to as `()` - however in C# this is a method invocation with no inputs.    
Since declaring this concept in c# fully is a bit verbose as we saw earlier with `Five` it is often omitted as an input.    

Example:
`WhatIsThis :: () -> ()`

In [ ]:
Unit WhatIsThis(Unit _) => unit;
// or better yet - which captures a bit of that functional syntax by omitting the unit input
Unit WhatIsThis() => unit;

// doesn't compile because we must return a unit
/*
Unit DoSomething() => 1 + 1; 
*/
Unit DoSomething() => ignore(1 + 1); // ignore is a wrapper to satisfy the c# compiler

// five could now be expressed as
Func<int> five = () => 5;
//or
int Five() => 5;

return (five(), Five()); // much better

Item1,Item2
5,5


#### Other useful types

In [ ]:
var tuples = ("Hello", 1);
Lst<int> list = List(1,2,3); // immutable, fixed size
Arr<int> arr = Array(1,2,3); // immutable, fixed size
Seq<int> seq = Seq(Range(1,10)); // similar to IEnumerable, but not infinite

Option<int> someInt = Some(1); // options wrap values that may be missing
Option<int> noInt = None;

### Currying
If 'functionaly' we can only have one input, how do we pass multiple inputs?    
1. tuples
2. currying! - a series of functions that each take one paramater    

Example: `PrintTwoParameters :: int -> int -> ()`

In [ ]:
Func<int, Func<int, Unit>> printTwoParameters = x => y => 
{
    Console.WriteLine($"x:{x}, y:{y}");
    return unit;
};

We now have a function that takes a single argument and returns a new function with that parameter baked in.  The new function also takes a single parameter.    

In [ ]:
var printOneThenY = printTwoParameters(1); // create a new func that prints 1 and whatever else you give it.
printOneThenY(2); // finish the function logic with the second parameter.
//or
printTwoParameters(1)(2);

x:1, y:2
x:1, y:2


Decalring functions the 'functional way' is a bit verbose and has a runtime cost in c#, so just declare them the 'classical way'

In [ ]:
static Unit PrintTwoParameters_classical(int x, int y)
{
    Console.WriteLine($"x={x}, y={y}");
    return unit;
}

PrintTwoParameters_classical(1,2);

x=1, y=2


Declaring functions "the classical way" misses out on the power of "curring"    
Language-ext has a prelude function to help restore this missing power.

In [ ]:
var printTwoParmeters_curry = curry<int, int, Unit>(PrintTwoParameters_classical);
var x = 1;
var y = 2;
var printOneThenY_curry = printTwoParmeters_curry(x);
var result  = printOneThenY_curry(y);
printTwoParmeters_curry(x)(y);

// or if input is a 'Func'
Func<int,int,Unit> printTwoParameters_classicalFunc = (x, y) => 
{
    Console.WriteLine($"x={x}, y={y}");
    return unit;
};
var alt = curry(printTwoParameters_classicalFunc); // input as 'Func' allows you to ommit type params

x=1, y=2
x=1, y=2


### Partial Application


Currying leads to 'partial application' - a powerful concept.    
The idea is to take a function of N parameters and apply some of the arguments, returning a new function of the remaining arguments.    
This is similar to `AdderGenerator` but more generalized.    

Example:    
`add :: int -> int -> int`    
partially apply 2 to get        
`addTwo :: int -> int`    

`isLessThan :: int -> int -> bool`    
partially apply 5 to get    
`isLessThan5 :: int -> bool`

In [ ]:
Func<int, int, int> add = (x, y) => x + y;
// partial application can be a bit messy in c# so language-ext helps out here.
Func<int,int> addTwo = par(add, 2); // partial application

WriteLine(addTwo(1));

Func<int, int, bool> isLessThan = (x, y) => x > y;
Func<int, bool> isLessThan5 = par(isLessThan, 5); 
Func<int, bool> isGreaterThan5 = lpar(isLessThan, 5);

WriteLine((isLessThan5(4), isGreaterThan5(4)));

// closer look at par and lpar
var isLessThan_fn = (int x) => (int y) => x > y; // a curry-able version of isLessThan, could also expressed as isLessThan_fn = curry(isLessThan);
Func<int, bool> isLessThan5_fn = isLessThan_fn(5); // partially apply 5
Func<int, bool> isGreaterThan5_fn = x => isLessThan_fn(x)(5); // partially apply 5 'from the left' - basically reverse par

WriteLine((isLessThan5_fn(4), isGreaterThan5_fn(4)));

// more examples


3
(True, False)
(True, False)


A few more examples    

Later on, we will revisit this example to show how much more powerful this can be.

In [ ]:
Func<Func<string, int, Unit>, int, int, int> adderWithPluggableLogger = (logger, x, y) =>  
{
    logger("x", x);
    logger("y", y);
    var result = x + y;
    logger("x+y", result);
    return result;
};

Func<string, int, Unit> consoleLogger = (name, val) => 
{
    Console.WriteLine($"{name}={val}"); 
    return unit;
};

var addWithConsoleLogging = par(adderWithPluggableLogger, consoleLogger);
var six = addWithConsoleLogging(2,4);

var addSixWithConsoleLogging = par(addWithConsoleLogging, six);
addSixWithConsoleLogging(3);

x=2
y=4
x+y=6
x=6
y=3
x+y=9


### Function Composition 
Taking the output of one function, and using it as the input of another function.    

For example, if you have a function that maps from T1 -> T2 and another function that maps from T2 -> T3, then you can compose them giving you a new function that maps from T1 -> T3.    

In [ ]:
// f(x)
Func<string, int> f = x => x.Length;
// g(x)
Func<int, bool> g = x => x < 3;

// a new function that uses the output of f(x) as the input of g(x)
// h(x) = g o f - that is g of f, or g after f
Func<string, bool> h = x => g(f(x));
//or using language-ext
Func<string, bool> h_composed = compose(f, g);
//language-ext alt
Func<string, bool> h_composed_alt = f.Compose(g);
//classical
bool h_classical (string x)
{
    var y = f(x);
    return g(y);
}


return h("hi");

True

### Combinators
A combinator is a function whos results only depends on the functions parameters. That is to say, a combinator is a function that only combines its parameters in some way.    
`compose` for example is a combinator, while `Console.WriteLine` is not.    
`compose` only uses its parameters, while `Console.WriteLine` depends on I/O.

**Combinator Birds**

In [ ]:
/// Identity function, or the Idiot bird
A I<A>(A x) => x;

/// The Kestrel
Func<B,A> K<A,B>(A x) => (B y) => x;

/// The Mockingbird
Func<A, A> M<A>(Func<A,A> x) => a => x(x(a));

/// The Thrush
Func<Func<A, B>, B> T<A, B>(A x) => (Func<A, B> y) => y(x);

/// The Queer bird
Func<Func<B, C>, Func<A, C>> Q<A, B, C>(Func<A, B> x) => (Func<B, C> y) => (A z) 
    => y(x(z));

/// The Starling
Func<Func<A, B>, Func<A, C>> S<A, B, C>(Func<A, B, C> x) => (Func<A, B> y) => (A z) =>
    x(z, y(z));

/// The infamous Y-combinator, or Sage bird
Func<A, B> Y<A, B>(Func<Func<A, B>, A, B> f) => (A x) =>
    f(Y<A, B>(f), x);

We will explore thes combinator mose closely when we use a different language such as Haskell, JS, or raw lambda calculus.  It is a bit messy in C#.    
That being said, here are a few cool things about these combinatos.    
`The Kestral`: This combinator is used to define "true", a const, or the first of two things.  It is also the same pattern you use when you create a fluent api in c# (returning yourself).    
`The Thrush`: applies x to y, this is the "pipe" opperator.    
`The Queer bird`: forward composition - 'f andThen g', where backards composition would ge 'f after g'.    
`Y-combinator or Sage bird`: Recursion!

A combinator is the safest form of a function. There is nothing outside of the function that can change its result.        
Using combitators from a combinator library is like using Lego from a lego set to create something new.    

Example:

In [ ]:
// combinator lib for parsing

#r "nuget:LanguageExt.Parsec"
using LanguageExt.Parsec;
using static LanguageExt.Parsec.Prim;
using static LanguageExt.Parsec.Char;
using static LanguageExt.Parsec.Expr;
using static LanguageExt.Parsec.Token;

Installed Packages LanguageExt.Parsec, 4.2.9

Lets parse some characters!

In [ ]:
var parser = ch('a'); // creates a parser that can read 'a'.
var result = parse(parser, "abcde");

return result.Reply.Result;

a

In [ ]:
var parser = satisfy(System.Char.IsLetter); // create a parser that can look for things based on a predicate, in this case all letters.
var aResult = parse(parser, "abcde");
var bResult = parse(letter, "bcdea"); // lang-ext built in letter parser

return (aResult.Reply.Result, bResult.Reply.Result);

Item1,Item2
a,b


Great! but this only reads one character.  What if we want to read more?

In [ ]:
var parser = many1(letter); // many1 will execute 'letter' 1 or more times and return a Seq
var result = parse(parser, "abcde");

return result.Reply.Result;

index,value
0,a
1,b
2,c
3,d
4,e


In [ ]:
var parser = many1(letter);
var result = parse(parser, "two words");

return result.Reply.Result; // oh no! it stopped after the space - it isn't a letter

index,value
0,t
1,w
2,o


In [ ]:
var spaces = many(satisfy(System.Char.IsWhiteSpace)); // many is 0 or more times
var word = from w in many1(letter)
           from s in spaces
           select w;

var result = parse(word, "two words");
return result.Reply.Result; // so far it doesn't seem any better, but since we handle spaces now lets try and apply our "word" parser "many" times.

index,value
0,t
1,w
2,o


In [ ]:
var parser = many1(word);
var result = parse(parser, "two words");

return result.Reply.Result;

index,value
0,"[ t, w, o ]"
1,"[ w, o, r, d, s ]"


In [ ]:
// and with a little adjustment to the inner parser to return the words rather than chars
var word = from w in asString(many1(letter))
           from s in spaces
           select w;
var parser = many1(word);
var result = parse(parser, "two words");

return result.Reply.Result;

index,value
0,two
1,words


So far it has been pretty easy to build up a parser from these existing peices.  But what about when we need to parse integers?

In [ ]:
var number = from d in asString(many1(digit))   // similar as before but we want a digit, not letter
             from s in spaces                   // check it out! we are reusing our spaces parser from before!
             select Int32.Parse(d);

var parser = many1(number);
var result = parse(parser, "123 1 2 3");

return result.Reply.Result;


index,value
0,123
1,1
2,2
3,3


Great! we can read numbers now! We are also starting to see the power of combinators in action.    
We can easily create and chain together new parsers that 'just work' because it is all Parser in Parser out.    
What about both letters and numbers at the same time?!

First we need a common abstraction for numbers and words

In [ ]:
abstract class Term { };

class Word : Term
{
    public readonly string Value;
    public Word(string value) => Value = value;
}

class Number : Term
{
    public readonly int Value;
    public Number(int value) => Value = value;
}

Next we need to introduce a new combinator: `either`.  This combinator takes two parsers and returns a new parser that will try both sub parsers if needed.    
For example: if a digit comes in, then the `word` parser will fail.  Should that happen, our new parer is now smart enough to try a second parser `digit`.

In [ ]:
// small updates to our word and number parsers to use the "terms" abstraction
var word = from w in asString(many1(letter))
           from s in spaces
           select new Word(w) as Term;

var number = from d in asString(many1(digit))
             from s in spaces
             select new Number(Int32.Parse(d)) as Term;

var term = either<Term>(word, number);

// create our final parser that removes leading white space and invokes `term` many times
var parser = from sp in spaces
             from ws in many1(term)
             select ws;

var result = parser.Parse("Hello 123");
return result.Reply.Result;

index,type,Value
0,Submission#110+Word,Hello
1,Submission#110+Number,123


Boom! We solved a simple problem (parsing a single character) and through the power of function values and combinators, built up a parser that can handle numbers and words! 

Lets keep going, we have one more problem to solve in this example.   
You may have noticed we always need `from s in spaces`.  As we add more parsable types this will get annoying.  Lets fix it!    

We will do this by creating one more parser `token`.  Its job will be to 'tokenize' the term, and by that I mean it will remove trailing spaces.

In [ ]:
Parser<A> token<A>(Parser<A> parserA) =>
    from res in parserA
    from spc in spaces
    select res;

easy peasy! lets use it!

In [ ]:
var word = from w in token(asString(many1(letter)))
           select new Word(w) as Term;

var number = from d in token(asString(many1(digit)))
             select new Number(Int32.Parse(d)) as Term;

var term = either<Term>(word, number);

// create our final parser that removes leading white space and invokes `term` many times
var parser = from sp in spaces
             from ws in many1(term)
             select ws;

var result = parser.Parse("   321Hello 123");
return result.Reply.Result;

index,type,Value
0,Submission#110+Number,321
1,Submission#110+Word,Hello
2,Submission#110+Number,123


"321Hello" was parsed as "321" and "Hello".  This may or may not be what we want. It happened this way because our spaces parser is looking for `0` or more spaces, and therefore succedded when there was no space.    

What if we wanted space delimited numbers and words but not words that contain numbers?

In [ ]:
// update spaces to utilize either
// we want to enforce `1` space - but can't fail on eof.
var spaces = either(
    eof,
    // we don't care about the spaces, and we need a common abstaction with eof parser
    // so we map to unit to create a parser<unit>
    many1(satisfy(System.Char.IsWhiteSpace)).Map(_ => unit) 
);

var word = from w in token(asString(many1(letter)))
           select new Word(w) as Term;

var number = from d in token(asString(many1(digit)))
             select new Number(Int32.Parse(d)) as Term;

var term = either<Term>(word, number);

var parser = from sp in spaces
             from ws in many1(term)
             select ws;

var result = parse(parser, "   4terms are here");
Console.WriteLine($"Faulted: {result.IsFaulted}, Result: {result.Reply.Error}"); // an error showing us what went wrong

var result2 = parse(parser, "   4 terms are here");
Console.WriteLine($"Faulted: {result2.IsFaulted}, Result: {result2.Reply.Result}"); // [number, word, word, word]
return result2.Reply.Result;

Faulted: False, Result: error at (line 1, column 19): unexpected end of stream, expecting letter or digit
Faulted: False, Result: [Submission#110+Number, Submission#110+Word, Submission#110+Word, Submission#110+Word]


index,type,Value
0,Submission#110+Number,4
1,Submission#110+Word,terms
2,Submission#110+Word,are
3,Submission#110+Word,here


Success! - sorta, we broke our parser in cases where there is no leading space.

In [ ]:
// perhaps we could just create a seperate spaces paser for when we are ok with `0` spaces
var maybeSpaces = many(satisfy(System.Char.IsWhiteSpace)); // perhaps isSpace = satisfy(System.Char.IsWhiteSpace) could be abstracted
var parser = from sp in maybeSpaces
             from ws in many1(term)
             select ws;

var result2 = parse(parser, "no leading space   7 terms are here");
return result2.Reply.Result;

index,type,Value
0,Submission#110+Word,no
1,Submission#110+Word,leading
2,Submission#110+Word,space
3,Submission#110+Number,7
4,Submission#110+Word,terms
5,Submission#110+Word,are
6,Submission#110+Word,here


Our final parsing code

In [ ]:
abstract record Term { };
record Word(string Value) : Term;
record Number(int Value) : Term;

var isSpace = satisfy(System.Char.IsWhiteSpace);
var spaces = either(eof, many1(isSpace).Map(_ => unit));

Parser<A> token<A>(Parser<A> parserA) =>
    from res in parserA
    from spc in spaces
    select res;

var word = 
    from w in token(asString(many1(letter)))
    select new Word(w) as Term;

var number = 
    from d in token(asString(many1(digit)))
    select new Number(Int32.Parse(d)) as Term;

var term = either<Term>(word, number);

var parser = 
    from sp in many(isSpace)
    from ws in many1(term)
    select ws;

var result = parse(parser, "   4terms are here");
Console.WriteLine($"Faulted: {result.IsFaulted}, Result: {result.Reply.Error}"); // an error showing us what went wrong

var result2 = parse(parser, "   4 terms are here");
Console.WriteLine($"Faulted: {result2.IsFaulted}, Result: {result2.Reply.Result}"); // [number, word, word, word]

var result3 = parse(parser, "no leading spaces   7 terms are here");
Console.WriteLine($"Faulted: {result3.IsFaulted}, Result: {result3.Reply.Result}"); // [word, word, word, number, word, word, word]
return result3.Reply.Result;

Faulted: True, Result: error at (line 1, column 5): unexpected t, expecting digit or end of input
Faulted: False, Result: [Number { Value = 4 }, Word { Value = terms }, Word { Value = are }, Word { Value = here }]
Faulted: False, Result: [Word { Value = no }, Word { Value = leading }, Word { Value = spaces }, Number { Value = 7 }, Word { Value = terms }, Word { Value = are }, Word { Value = here }]


index,type,Value
0,Submission#116+Word,no
1,Submission#116+Word,leading
2,Submission#116+Word,spaces
3,Submission#116+Number,7
4,Submission#116+Word,terms
5,Submission#116+Word,are
6,Submission#116+Word,here


I'm not very familure with query syntax - What would this look like in method syntax?

In [ ]:
abstract record Term { };
record Word(string Value) : Term;
record Number(int Value) : Term;

var isSpace = satisfy(System.Char.IsWhiteSpace);
var spaces = either(eof, many1(isSpace).Map(_ => unit));

Parser<A> token<A>(Parser<A> parserA) =>
    parserA
        .SelectMany(
            inp => spaces, 
            (res, sp) => res
        );

var word = 
    token(asString(many1(letter)))
        .Map(w => new Word(w) as Term);

var number = 
    token(asString(many1(digit)))
        .Map(d => new Number(Int32.Parse(d)) as Term);

var term = either<Term>(word, number);

var parser = 
    many(isSpace)
        .SelectMany(
            inp => many1(term), 
            (sp, ws) => ws
        );

var result = parse(parser, "   4terms are here");
Console.WriteLine($"Faulted: {result.IsFaulted}, Result: {result.Reply.Error}"); // an error showing us what went wrong

var result2 = parse(parser, "   4 terms are here");
Console.WriteLine($"Faulted: {result2.IsFaulted}, Result: {result2.Reply.Result}"); // [number, word, word, word]

var result3 = parse(parser, "no leading spaces   7 terms are here");
Console.WriteLine($"Faulted: {result3.IsFaulted}, Result: {result3.Reply.Result}"); // [word, word, word, number, word, word, word]
return result3.Reply.Result;

Faulted: True, Result: error at (line 1, column 5): unexpected t, expecting digit or end of input
Faulted: False, Result: [Number { Value = 4 }, Word { Value = terms }, Word { Value = are }, Word { Value = here }]
Faulted: False, Result: [Word { Value = no }, Word { Value = leading }, Word { Value = spaces }, Number { Value = 7 }, Word { Value = terms }, Word { Value = are }, Word { Value = here }]


index,type,Value
0,Submission#117+Word,no
1,Submission#117+Word,leading
2,Submission#117+Word,spaces
3,Submission#117+Number,7
4,Submission#117+Word,terms
5,Submission#117+Word,are
6,Submission#117+Word,here


Not bad, personally I think I prefer the query expression syntax.  It would be worth getting more familure with it.  As things get more complex, it seems easier to read and think about than the method syntax.

### Function Signatures

Documentation only goes so far and gets stale. Type systems can help you communicate with compile time checking.    
Strive for declarative signatures.

Example:
```csharp
// Create Date takes 3 ints... but in what order?
DateTime CreateDate(int a, int b, int c);
// Ok - year, month, day... but any int?
DateTime CreateDate(int year, int month, int day);
// Oh Ok! A valid year, month, day
DateTime CreateDate(Year year, Month month, Day day);
```

Creating a type for every invidiual usage of int would be insane!  What does Year, Month, Day really look like?    

Language-Ext has some generic types to help create new constrained typed:

In [ ]:
public class Year : NumType<Year, TInt, int>
{
    Year(int x) : base(x) { }
}

public class Month : NumType<Month, TInt, int>
{
    Month(int x) : base(x) { }
}

public class Day : NumType<Day, TInt, int>
{
    Day(int x) : base(x) { }
}

var day = Day.New(1);
var month = Month.New(1);
var year = Year.New(2000);

Great! A day is not a month, but we can still have a day of 100 in the month of 20...    

Time for some constrains:

constructors with exceptions

In [ ]:
public class Year : NumType<Year, TInt, int>
{
    Year(int x) : base(x) 
    { 
       if(x < 1970 || x > 2050) throw new ArgumentException("Invalid year");
    }
}

public class Month : NumType<Month, TInt, int>
{
    Month(int x) : base(x)
    { 
       if(x < 1 || x > 12) throw new ArgumentException("Invalid month");
    }
}

public class Day : NumType<Day, TInt, int>
{
    Day(int x) : base(x)
    { 
       if(x < 1 || x > 31) throw new ArgumentException("Invalid day");
    }
}

Or - Declarative Types

In [ ]:
using LanguageExt.ClassInstances;
using LanguageExt.ClassInstances.Const;
using LanguageExt.ClassInstances.Pred;

public class Year : NumType<Year, TInt, int, Range<TInt, int, I1970, I2050>>
{
    Year(int x) : base(x) { }
}

public class Month : NumType<Month, TInt, int, Range<TInt, int, I1, I12>>
{
    Month(int x) : base(x) { }
}

public class Day : NumType<Day, TInt, int, Range<TInt, int, I1, I31>>
{
    Day(int x) : base(x) { }
}

The Declarative approach may take some getting used to, but you can look at the type signature and see everything you need to know. Its hard to go wrong, and you have the type system working for you.    
Your validators can be reusable, and you can easily navigate to the validators in you IDE.

Consider the following:

In [ ]:
int DoSomethingWithAList(Lst<int> list)
{
    if(list.IsEmpty) throw new ArgumentException("List must not be empty");
    return list.Count;
}

DoSomethingWithAList(Lst<int>.Empty);

Error: System.ArgumentException: List must not be empty
   at Submission#121.DoSomethingWithAList(Lst`1 list)
   at Submission#121.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

VS

In [ ]:
int DoSomethingWithAList(Lst<NonEmpty, int> list)
{
    // List can't ever be empty here
    return list.Count;
}


DoSomethingWithAList(Lst<int>.Empty);

Error: System.ArgumentException: List must not be empty
   at Submission#121.DoSomethingWithAList(Lst`1 list)
   at Submission#122.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

Same end result - But! NonEmpty is reusable and communicating to the caller through the type system. Nifty!

After all this, what if we still manage to pass an invalid date, what does this do?
```csharp
string CreateDateString(Year year, Month month, Day day);
```
Is string null? exception thrown?  How do we communicate that?    


Time for a new word - Enter the Monad!    
Monads help you get rid of repetative boilerplate, and provide a safe wrapper for our values to live in.    
We will get to a better defenition, but for now let's look at an example:

In [ ]:
Try<string> CreateDateString(Year year, Month month, Day day) => () =>
        new DateTime(year.Value, month.Value, day.Value).ToShortDateString();

What is happening here?    
First, we can see that the return type is now trying to tell us something.  `Try<A>` - perhaps this could fail then?    
Next, you should notice it is a delegate - we are returning a new func `=> () =>`    
This is done so we can wrap the exception based validation happening in `DateTime` and work with a safe value in all cases.    
Lets use it!

In [ ]:
Try<string> date = CreateDateString(Year.New(2000), Month.New(2), Day.New(30));
return date.IfFail("Invalid date");

Invalid date

Sweet! What else can we do with a try? - Pattern Matching!

In [ ]:
Try<string> date = CreateDateString(Year.New(2000), Month.New(2), Day.New(30));
Try<string> date2 = CreateDateString(Year.New(2000), Month.New(2), Day.New(14));
var getMessage = (string dt) => $"The date is {dt}";
var getErrorMessage = (Exception ex) => $"Invalid date ({ex.Message})";

var invalidDate = date.Match(
    Succ: getMessage,
    Fail: getErrorMessage
    );
var validDate = date2.Match(
    Succ: getMessage,
    Fail: getErrorMessage
    );
return (invalidDate, validDate);

Item1,Item2
"Invalid date (Year, Month, and Day parameters describe an un-representable DateTime.)",The date is 2/14/2000


Nice - we either have a good value to process, or a useful default.    
But why? - you can't process the message until you know there is something there.  goodbye null refs!

Lets look at a more complicated example:

Here we will create a DateTime and a simple calculation on the dates.

In [ ]:
public static Try<DateTime> ToDate(this (Year year, Month month, Day day) date) => () =>
    new DateTime(date.year.Value, date.month.Value, date.day.Value);

Try<DateTime> fromDate = (Year.New(2000), Month.New(1), Day.New(1)).ToDate();
Try<DateTime> toDate = (Year.New(2017), Month.New(5), Day.New(20)).ToDate();

TimeSpan result = fromDate.Match(
    Succ: begin =>
        toDate.Match(
            Succ: end => end - begin,
            Fail: ex  => TimeSpan.Zero),
    Fail: ex => TimeSpan.Zero);

Console.WriteLine($"{result}")

6349.00:00:00


... That isn't super great to look at.  It isn't bad, but what if we ended up with more nested lambdas?  Ahh!!!

Remember the query expression we saw earlier?  They are C#'s way of doing mondas - our new fancy word.  So how can that clean this up?

In [ ]:
var result = 
    from begin in fromDate
    from end in toDate
    select end - begin;

result.IfSucc(r => Console.WriteLine($"{r}"));

6349.00:00:00


Way easier to read, and you may have noticed it also keeps us in the monadic context.  It is best to stay monadic until you really need a value.

What if we shouldn't ever have an exception, but we may not have the value we want?

In [ ]:
Option<int> ParseInt(string value) =>
    Int32.TryParse(value, out var result)
        ? Some(result)
        : None;

Now we either have a good value, or a safe way of saying we don't have a good default for this. - You may be wondering, how is 'None' different than 'null'? it is nothing after all.    
Well - `null` is actually nothing and we can't work with that. On the other hand `None` is a value, just like `Some(value)` is.

In [ ]:
var calculate = (string x, string y, string z) => 
    from a in ParseInt(x)
    from b in ParseInt(y)
    from c in ParseInt(z)
    select a + b + c;

var valid = calculate("10", "20", "30");
var invalid =  calculate("invalid number", "10", "20");
return (valid, invalid);

Item1,Item2
[ 60 ],[ ]


No weird assumed default, no exceptions, no null refs - just values.  And we avoided nested `if` hell!

So why are monads cool?  Here is the alternative - you tell me

In [ ]:
int? ParseInt(string value) =>
    Int32.TryParse(value, out var result)
        ? result
        : null;

int? Calculate(string a, string b, string c)
{
    var aNum = ParseInt(a);
    if (aNum is not null)
    {
        var bNum = ParseInt(b);
        if (bNum is not null)
        {
            var cNum = ParseInt(c);
            if (cNum is not null)
            {   
                return aNum + bNum + cNum;
            }
        }
    }
    return null;
}

var valid = Calculate("10", "20", "30");
var invalid = Calculate("invalid number", "10", "20");
return (valid, invalid); // return null gross!

Item1,Item2
60,<null>


### What is a monad anyway?

Monads come from category theory, and a few basic ways to define them are:
- Design patterns that eliminate error prone boilerplate
- Encapsulated functionality (with a couple rules)

C#'s LINQ is actually an example of monads. Let's take a closer look at what they are.   

the `IEnumerable<A>` monad frees you from writing this looping behavior
```csharp
foreach(var a in listA)
{
    foreach(var b in listB)
    {
        yield return Process(a, b);
    }
}
```
and instead allows you to write something like this
```csharp
var result = 
    from a in listA
    from b in listB
    select Process(a, b);
```

This is simple example, and may seem like a small win - but we have traded statements for expressions!

Lets take another look at the `Option` monad we introduced in the last section and see what error prone boilerplate it can remove.


In [ ]:
using static LanguageExt.Prelude;

Option<string> optionA = Some("Hello, ");
Option<string> optionB = Some("World");
Option<string> optionNone = None;

var result1 = 
    from x in optionA
    from y in optionB
    select x + y;

var result2 = 
    from x in optionA
    from y in optionB
    from z in optionNone
    select x + y + z;

return (result1, result2);

Item1,Item2
"[ Hello, World ]",[ ]


Result1 succesfully concatinated two strings and shows us "Hello, World".    
Result2 failed because there was a bad string value and shows us "".    

So, what did `Option` hide?

In [ ]:
string valueA = "Hello, ";
string valueB = "World";
string valueC = null;

string result1 = null;
if(valueA != null)
{
    if(valueB != null)
    {
       result1 = valueA + valueB; 
    }
}

string result2 = null;
if(valueA != null)
{
    if(valueB != null)
    {
        if(valueC != null)
        {
           result2 = valueA + valueB + valueC; 
        }
    }
}

return (result1, result2);

Item1,Item2
"Hello, World",<null>


Similar results - though that pesky `null` made a return! :(    
    
What else though?    
We must manualy check for good values before our calculation using if statements    
We may have introduced a bug 
- we cold have accidentally calculated `result1 = valueA + valueB + valueC;` which would be impossible in our first example. 
- we have no `else` and are not accounting for when the value is `null`, whad do wi actually want in this case?      

We have introduced a higher Cyclomatic Complexity (even if you refactor nested ifs to &&'s)    

What does this mean?
```csharp
from a in ma
```
This is saying "Get the value a out of the monad ma".    
If the monad is IEnumerable then its the value in the stream.    
If the monad is Option then its the value if it is not in a None state.    

So how does this work?
```csharp
from a in ma
from b in mb
```
This is saying "If we got a value a, then get the value b out of mb".  
So if there was no value for ma, then we never try to get b out of mb.    
This is capturing the if behaviour from before. The process is known as *binding*.

finally we `select` - this means "put the value a + b back into a new monad".
```csharp
from a in ma
from b in mb
select a + b;
```
For IEnumerable this would mean "create a new stream of values to put the result in"    
For Option this would mean "create a new Option and put the results in"


In C# `Select` and `SelectMany` make a type monadic.

#### The Monad rules

return: `a -> ma` (not c# return, think constructor)    
bind: `ma -> (a -> mb) -> mb`

So - a monad can put a value inside, and bind to another monad